### 数据转换
* train分成4：1，4为train_ffm,1为vali_ffm
* test为测试集

In [5]:
import pandas as pd
from pandas import get_dummies
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import numpy as np
import os
import gc
import xlearn as xl
import math

In [6]:
class FFMFormat:
    def __init__(self,vector_feat,one_hot_feat,continus_feat):
        self.field_index_ = None
        self.feature_index_ = None
        self.vector_feat=vector_feat
        self.one_hot_feat=one_hot_feat
        self.continus_feat=continus_feat
        
    def get_params(self):
        pass

    def set_params(self, **parameters):
        pass

    def fit(self, df, y=None):
        self.field_index_ = {col: i for i, col in enumerate(df.columns)}
        self.feature_index_ = dict()
        last_idx = 0
        for col in df.columns:
            if col in self.one_hot_feat:
                print(col)
                df[col]=df[col].astype('int')
                vals = np.unique(df[col])
                for val in vals:
                    if val==-1: continue
                    name = '{}_{}'.format(col, val)
                    if name not in self.feature_index_:
                        self.feature_index_[name] = last_idx
                        last_idx += 1
            elif col in self.vector_feat:
                print(col)
                vals=[]
                for data in df[col].apply(str):
                    if data!="-1":
                        for word in data.strip().split(' '):
                            vals.append(word)
                vals = np.unique(vals)
                for val in vals:
                    if val=="-1": continue
                    name = '{}_{}'.format(col, val)
                    if name not in self.feature_index_:
                        self.feature_index_[name] = last_idx
                        last_idx += 1
            self.feature_index_[col] = last_idx
            last_idx += 1
        return self

    def fit_transform(self, df, y=None):
        self.fit(df, y)
        return self.transform(df)

    def transform_row_(self, row):
        ffm = []

        for col, val in row.loc[row != 0].to_dict().items():
            if col in self.one_hot_feat:
                name = '{}_{}'.format(col, val)
                if name in self.feature_index_:
                    ffm.append('{}:{}:1'.format(self.field_index_[col], self.feature_index_[name]))
                # ffm.append('{}:{}:{}'.format(self.field_index_[col], self.feature_index_[col], 1))
            elif col in self.vector_feat:
                for word in str(val).split(' '):
                    name = '{}_{}'.format(col, word)
                    if name in self.feature_index_:
                        ffm.append('{}:{}:1'.format(self.field_index_[col], self.feature_index_[name]))
            elif col in self.continus_feat:
                if val!=-1:
                    ffm.append('{}:{}:{}'.format(self.field_index_[col], self.feature_index_[col], val))
        return ' '.join(ffm)

    def transform(self, df):
        # val=[]
        # for k,v in self.feature_index_.items():
        #     val.append(v)
        # val.sort()
        # print(val)
        # print(self.field_index_)
        # print(self.feature_index_)
        return pd.Series({idx: self.transform_row_(row) for idx, row in df.iterrows()})

In [7]:
one_hot_feature=['LBS','age','carrier','consumptionAbility','education','gender','advertiserId','campaignId', 'creativeId',
       'adCategoryId', 'productId', 'productType']
vector_feature=['interest1','interest2','interest5','kw1','kw2','topic1','topic2','os','ct','marriageStatus']
continus_feature=['creativeSize']

path = 'data1'
train_name = 'train.csv'
test_name = 'test.csv'
temp_name = 'ffm.csv'
train_ffm_name = 'train_ffm.csv'
vali_ffm_name = 'vali_ffm.csv'
test_ffm_name = 'test_ffm.csv'

In [8]:
tr = FFMFormat(vector_feature,one_hot_feature,continus_feature)
train = pd.read_csv(path+'/'+train_name)
test = pd.read_csv(path+'/'+test_name)

Y = np.array(train.pop('label'))
len_train = len(train)
len_vali = math.ceil(len_train/5)

data = pd.concat([train,test],ignore_index=True)

data = data[one_hot_feature+vector_feature+continus_feature]
# vali = train[one_hot_feature+vector_feature+continus_feature]

tr = FFMFormat(vector_feature,one_hot_feature,continus_feature)
data_ffm = tr.fit_transform(data)
data_ffm.to_csv(path+'/'+temp_name,index=False)



LBS
age
carrier
consumptionAbility
education
gender
advertiserId
campaignId
creativeId
adCategoryId
productId
productType
interest1
interest2
interest5
kw1
kw2
topic1
topic2
os
ct
marriageStatus


In [9]:
with open(path+'/'+temp_name) as fin:
    f_train_out = open(path+'/'+train_ffm_name,'w')
    f_vali_out = open(path+'/'+vali_ffm_name,'w')
    f_test_out = open(path+'/'+test_ffm_name,'w')
    for (i,line) in enumerate(fin):
        if i < len_vali:
            f_vali_out.write(str(Y[i])+' '+line)
        elif i < len_train:
            f_train_out.write(str(Y[i])+' '+line)
        else:
            f_test_out.write(line)
    f_train_out.close()
    f_vali_out.close()
    f_test_out.close()
    print('转换成功，数据位置：'+path)

del train
del test
del data
del data_ffm
gc.collect()

转换成功，数据位置：data1


5721

### 模型训练

In [10]:
import pandas as pd
from pandas import get_dummies
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import numpy as np
import os
import gc
import xlearn as xl
import time

In [11]:
path = 'data1'
test_name = 'test.csv'
train_ffm_name = 'train_ffm.csv'
vali_ffm_name = 'vali_ffm.csv'
test_ffm_name = 'test_ffm.csv'
submission_name = 'submission.csv'

model_name = path+"/model.out"
output_name = path+"/output.txt"

In [12]:
param = {'task':'binary', 'lr':0.02, 'lambda':0.000001,'metric': 'auc','opt':'ftrl','epoch':4,'k':2,
    'alpha': 1.5, 'beta': 1, 'lambda_1': 0.0, 'lambda_2': 0.0}
ffm_model = xl.create_ffm()
ffm_model.setTrain(path+'/'+train_ffm_name)
ffm_model.setValidate(path+'/'+vali_ffm_name)
ffm_model.fit(param,model_name)
del ffm_model
gc.collect()

test.csv use time : 4


### 模型预测
* 模型导出为model_name

In [ ]:
s_1 = time.time()
ffm_model.setTest('%s/%s'%(path,test_ffm_name))
ffm_model.setSigmoid()
ffm_model.predict(model_name,output_name)
sub = pd.DataFrame()
test_df = pd.read_csv('%s/%s'%(path,test_name))
sub['aid']=test_df['aid']
sub['uid']=test_df['uid']
sub['score'] = np.loadtxt(output_name)
sub.to_csv('%s/%s'%(path,submission_name),index=False)
if os.path.exists(output_name):
    os.remove(output_name)
del test_df
del ffm_model
del sub
gc.collect()
s_2 = time.time()
print('%s use time : %d'%(test_name,(s_2-s_1)))